# Training a Convolutional Neural Network (CNN) for Bike Frame Model Detection

In [5]:
#!pip install scikit-learn
#!pip install tensorflow
#!pip install pillow
#!pip install tqdm




## Importing

In [6]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
from PIL import Image


## Data Collection and Preprocessing
- I collected data from google for the most commonly used bike frames on the market
- Steps taken to preprocess the data

In [8]:
# Progress tool
class TrainingCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"End of epoch {epoch}. Loss: {logs['loss']}, Accuracy: {logs['accuracy']}")

dataset_dir = 'Data'
output_format = 'JPEG'

# def convert_images(dataset_dir, output_dir, output_format):
#     # Get a list of all files in the directory
#     all_files = [os.path.join(root, file) for root, dirs, files in os.walk(dataset_dir) for file in files]
    
#     # Initialize a progress bar
#     pbar = tqdm(total=len(all_files), ncols=70)

#     for file_path in all_files:
#         try:
#             if file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
#                 image = Image.open(file_path)
#                 # Ensure the image is in RGB mode
#                 if image.mode != 'RGB':
#                     image = image.convert('RGB')
#                     # Change the file extension to output_format
#                     rel_path = os.path.relpath(file_path, dataset_dir)
#                     new_file_path = os.path.join(output_dir, os.path.splitext(rel_path)[0] + '.' + output_format.lower())
#                     os.makedirs(os.path.dirname(new_file_path), exist_ok=True)
#                     image.save(new_file_path, output_format)
#                     print(f"Converted file {file_path} to {new_file_path} in RGB mode.")
#             else:
#                 print(f"Skipping unsupported file format: {file_path}")
#         except Exception as e:
#             print(f"Error processing file {file_path}: {e}")
        
#         # Update the progress bar
#         pbar.update(1)

#     # Close the progress bar when done
#     pbar.close()

# output_dir = 'processed_data'
# convert_images(dataset_dir, output_dir, output_format)


#directories
data_dir = 'Data'
batch_size = 32
img_height = 180
img_width = 180

# training data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# validation data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 3268 files belonging to 14 classes.
Using 2615 files for training.
Found 3268 files belonging to 14 classes.
Using 653 files for validation.


## Create Model

In [9]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_ds.class_names))
])


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Training the Model

In [10]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  callbacks=[TrainingCallback()]
)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_1490]

## Model Evaluation and Results

In [ ]:
# Make predictions on the validation set
val_images, val_labels = next(iter(val_ds))
val_predictions = model.predict(val_images)

# Convert predictions from probabilities to class labels
val_predictions = tf.argmax(val_predictions, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print(f'Accuracy: {accuracy}')

# Calculate F1 score
f1 = f1_score(val_labels, val_predictions, average='weighted')
print(f'F1 Score: {f1}')

## Saving the Model

In [ ]:
# Save the model in TensorFlow SavedModel format
model.save('my_model')

In [12]:
import tensorflowjs as tfjs
model = tf.keras.models.load_model('Bike_model')

# Convert the model to TensorFlow.js Layers format 
tfjs.converters.save_keras_model(model, 'tfjs_model')


  0%|                       | 6/3271 [2:14:00<1215:26:40, 1340.15s/it]
/Users/matthewreynolds/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
